**mAP_Second_Try**

대회의 메트릭과 매우 유사한 mAP를 이용하여 score를 계산해보자.

In [ ]:
import numpy as np
import pandas as pd

우리가 예측(Prediction)한 submission 파일을 불러온다.

In [ ]:
oof_preds0 = pd.read_csv('./root_dir/submission.csv')

In [ ]:
oof_preds0.head(10)

우리가 예측한 것의 정답인, label이 붙어 있는 정답지를 불러온다.

In [ ]:
df = pd.read_csv('./input/testing_file_20181205.csv')
df.head()

In [ ]:
df.head(10)

1. Csv 파일 Parsing

     (1) 정답지에 bbox_target이라는 열을 만들어서, x, y, w, h로 흩어져 있던 열들을 합친다.

In [ ]:
df['bbox_target'] = (df['x'].astype(str) +
                    ' ' + 
                    df['y'].astype(str) +
                    ' ' +
                    df['width'].astype(str) +
                    ' ' +
                    df['height'].astype(str))

In [ ]:
df.loc[:, 'bbox_target'] = df.loc[:, 'bbox_target'].map(lambda x: x.split(' '))

In [ ]:
df.head(10)

     (2) patientId와 bbox_target만 남긴 행열을 만든다. 

In [ ]:
df = df.groupby(['patientId'], as_index = False)['bbox_target'].agg('sum')  

In [ ]:
df.head(10)

     (3) 우리의 예측 값인 PredictionString을 정답지 파일에 merge 하여 bbox_pred 열로 만든다.

In [ ]:
df = df.merge(oof_preds0, on = 'patientId', how = 'right') # modify left or right

In [ ]:
df = df.fillna('')
df.head(10)

In [ ]:
df.loc[:, 'bbox_pred'] = (oof_preds0.loc[:, 'PredictionString']) 
#원래의 코드에서 바꾼 부분.
#원래의 코드는 우리가 예측한 csv 값을 df의 prediction String열에 머지한 후 df의 bbox_pred에 넣는 거였다.
#수정한 코드는 우리가 예측한 csv 값을 바로 df의 bbox_pred에 넣는 것이므로 사실상 같은 코드이다!
df.head(10)

In [ ]:
df = df.drop(['PredictionString'], axis=1)
df.head(10)

     (4) bbox_pred 열의 값들을, 이용하기 좋게 파싱한다. (띄어쓰기 된 것을 ','로 나누도록 한다.)

In [ ]:
df.loc[:, 'bbox_pred'] = df.loc[:, 'bbox_pred'].str.strip()

In [ ]:
df = df.fillna(' ')
df.loc[:, 'bbox_pred'] = df.loc[:, 'bbox_pred'].map(lambda x: x.split(' '))

In [ ]:
df.head(10)

     (5) bbox_pred의 score 부분만 얻어서 bbox_scores라는 열을 만든다.

In [ ]:
def parse_scores(x):
    if len(x)!=2: # 값의 길이가 '2'가 아니면, 즉 [,]가 아니면!
        scores = [x[k] for k in range(0,len(x),5)]
        for score in range(len(scores)):
            scores[score] = float(scores[score])
        return np.asarray(scores)

In [ ]:
df.loc[:, 'bbox_scores'] = df.loc[:, 'bbox_pred'].map(parse_scores)

In [ ]:
df.head(10)

     (6) bbox_pred 대신 bbox_preds라는 열을 만들어서, score 부분을 뺀 좌표 값들만 넣는다. bbox_pred는 지운다.

In [ ]:
def parse_bbox(x):
    if len(x)!=2:
        bbox = [int(x[k]) for k in range(0,len(x)) if k%5 != 0]
        return np.asarray(bbox).reshape(int(len(bbox)/4),4)

In [ ]:
df.loc[:, 'bbox_preds'] = df.loc[:, 'bbox_pred'].map(parse_bbox)

In [ ]:
df = df.drop(['bbox_pred'], axis=1)

In [ ]:
df.head(10)

     (7) bbox_scores와 bbox_preds의 None 값을 빈 값으로 바꿔준다.

In [ ]:
df.loc[df['bbox_scores'].isnull(),['bbox_scores']] = df.loc[df['bbox_scores'].isnull(),'bbox_scores'].apply(lambda x: np.asarray([]))

In [ ]:
df.loc[df['bbox_preds'].isnull(),['bbox_preds']] = df.loc[df['bbox_preds'].isnull(),'bbox_preds'].apply(lambda x: np.asarray([]))

In [ ]:
df.head(10)

     (8) nan 값들을 빈 값으로 바꾸고, 여러 값들을 map를 구하기 쉽게 파싱해준다.

In [ ]:
def parse_target_str(x):
    if x[0] != 'nan':
        bbox = np.asarray([int(float(x[k])) for k in range(0,len(x))])
        return bbox.reshape(int(len(bbox)/4),4)

In [ ]:
df.loc[:,'bbox_target'] = df.loc[:,'bbox_target'].map(parse_target_str)

In [ ]:
df.loc[df['bbox_target'].isnull(),['bbox_target']] = df.loc[df['bbox_target'].isnull(),'bbox_target'].apply(lambda x: np.asarray([]))

In [ ]:
df.head(10)

2. mAP

reference1: https://www.kaggle.com/chenyc15/mean-average-precision-metric and edited herein
reference2: https://www.kaggle.com/cchadha/mean-average-precision-iou-on-cnn-oof-preds

RSNA Competition은 IoU 임계치에 따른 mAP를 구해서 submission.csv 평가한다. IoU는 두 bounding box가 일치하는 정도를 나타내며 IoU 각 임계치(0.4 to 0.75까지 0.05 단위, (0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75))에서 mAP를 구한다.

1.  Bbox, image, model과 AP, mAP 관계
 - bounding box 1개는 AP 1개 출력
 - image 1개는 mAP 1개 출력
 - 모든 사진의 mAP평균을 낸 값으로 모델의 mAP를 계산

2. AP계산방법
 - ground truth와 prediction box 둘 다 값이 없으면 그냥 패스
 -  ground truth와 prediction box 둘 중 하나가 0이면 그 image의 FP가 0이므로 AP도 0이므로 return 0
 - ground truth와 prediction box 모두 양수이면 prediction box들을 score에 대해 내림차순으로 정렬한 후 차례대로 truth box랑 매칭해서 tp,tn,fp 구함.
  * (iou> threshold) ->  TP(정탐)
  * (iou< threshold) ->  FP(미탐)
  * (예측한 bbox - 찾은 bbox) -> FN(오탐)
  * 이후, 각 threshold에서 (tp/tp+fp+tn) 구한 후 thresholds 개수로 나눠서 AP 구한다.

3. 기타
 - iou()는 두개의 바운딩 박스를 비교해서 일치정도를 계산하는 함수이다.
 - map_iou()는 RSNA Validation 방식을 코드로 구현한 함수이다.

In [ ]:
# helper function to calculate IoU
def iou(box1, box2):
    box1 = box1.astype(np.float)
    box2 = box2.astype(np.float)
    x11, y11, w1, h1 = box1
    x21, y21, w2, h2 = box2
    assert w1 * h1 > 0
    assert w2 * h2 > 0
    x12, y12 = x11 + w1, y11 + h1
    x22, y22 = x21 + w2, y21 + h2

    area1, area2 = w1 * h1, w2 * h2
    xi1, yi1, xi2, yi2 = max([x11, x21]), max([y11, y21]), min([x12, x22]), min([y12, y22])
    
    if xi2 <= xi1 or yi2 <= yi1:
        return 0
    else:
        intersect = (xi2-xi1) * (yi2-yi1)
        union = area1 + area2 - intersect
        return intersect / union

In [ ]:
#thresholds 값은 폐렴을 진단할때 validation의 기본 thresholds 값을 이용했다.
# 한 사진에 대해서 mAP 값을 구하는 함수이다.
def map_iou(boxes_true, boxes_pred, scores, thresholds = [0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75]):
    """
    Mean average precision at differnet intersection over union (IoU) threshold
    
    input:
        boxes_true: Mx4 numpy array of ground true bounding boxes of one image. 
                    bbox format: (x1, y1, w, h)
                    ground truth BBox 배열 or 배열들 (x, y, w, h) 
        boxes_pred: Nx4 numpy array of predicted bounding boxes of one image. 
                    bbox format: (x1, y1, w, h)
                    예측한 bbox 배열 or 배열들 (x, y, w, h)
        scores:     length N numpy array of scores associated with predicted bboxes
        thresholds: IoU shresholds to evaluate mean average precision on
    output: 
        map: mean average precision of the image
    """
    
    # According to the introduction, images with no ground truth bboxes will not be 
    # included in the map score unless there is a false positive detection (?)
        
    # return None if both are empty, don't count the image in final evaluation (?)
    """
    모두 0이면 dont count
    하나는 0이고 하나는 0이 아니면, TP가 0이므로 return 0을 한다.
    둘 다 하나 이상의 값이 있으면 TP, FN, FP 계산이 필요하다.
    """
    if len(boxes_true) == 0 and len(boxes_pred) == 0:     
        return None
    elif len(boxes_true) == 0 and len(boxes_pred) > 0:  
        return 0
    elif len(boxes_true) > 0 and len(boxes_pred) == 0:
        return 0
    elif len(boxes_true) > 0 and len(boxes_pred) > 0:
        assert boxes_true.shape[1] == 4 or boxes_pred.shape[1] == 4, "boxes should be 2D arrays with shape[1]=4"
        if len(boxes_pred):
            assert len(scores) == len(boxes_pred), "boxes_pred and scores should be same length"
            # sort boxes_pred by scores in decreasing order, bbox를 scores에 따른 내림차순 정렬 
            boxes_pred = boxes_pred[np.argsort(scores)[::-1], :] 

        map_total = 0
        map_threshold = []

        # loop over thresholds
        for t in thresholds:
            matched_bt = set()
            tp, fn = 0, 0
            for i, bt in enumerate(boxes_true):
                matched = False
                for j, bp in enumerate(boxes_pred):
                    miou = iou(bt, bp)
                    if miou >= t and not matched and j not in matched_bt:
                        matched = True # IoU가 t임계치를 넘었으므로 정탐
                        tp += 1 # bt is matched for the first time, count as TP
                        matched_bt.add(j)
                if not matched: # 모든 bbox t임계치를 넘지 못했으므로 미탐
                    fn += 1 # bt has no match, count as FN
                    
            # 예측한 bbox 중에 정탐한 bbox를 제외하면 나머지 bbox는 오탐으로 간주
            fp = len(boxes_pred) - len(matched_bt) # FP is the bp that not matched to any bt
            m = tp / (tp + fn + fp)
            map_threshold.append(m) # AP per each thresholds
            map_total += m # 모든 임계치에 대해서 m 값을 계산

        map_total=map_total / len(thresholds)
        map_threshold.append(map_total) #kaggle validation function
    
    return map_threshold

In [ ]:
#상위 10개만 테스트
for row in range(10):
    print(map_iou(df['bbox_target'][row], df['bbox_preds'][row], df['bbox_scores'][row]))

In [ ]:
#non이 아닐 때만, map_iou 계산
map_scores = [x for x in [map_iou(df['bbox_target'][row], df['bbox_preds'][row], df['bbox_scores'][row]) for row in range(len(df))] if x is not None]

In [ ]:
#map 값
np.mean(map_scores)

In [ ]:
# if mAP is 0, then change the 0 to [0, 0, 0, 0, 0, 0, 0, 0, 0]
for i in range(len(map_scores)):
    if map_scores[i] == 0 :
        del map_scores[i]
        map_scores.insert(i,[0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
# reverse list, [i][j] -> [j][i]
new_list = list(map(list, zip(*map_scores)))

In [ ]:
thres_list = []
for i in range(len(new_list)):
    thres_list.append(np.mean(new_list[i])) # AP per thresholds

In [ ]:
thres_list

In [ ]:
def AP_N(thres, thres_list):
    thres_sum = []
    thresholds = [0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75]
    index = thresholds.index(thres)
    for i in range(index, len(thresholds)):
        thres_sum.append(thres_list[i])
    
    return np.mean(thres_sum)           # mAP    

In [ ]:
thresholds = [0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75]

for i in range(len(thresholds)):
    print(AP_N(thresholds[i], thres_list))